In [ ]:
import random

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import shapely.geometry as geo
import shapely.affinity as aff
from descartes.patch import PolygonPatch

In [ ]:
random.seed(1618)

In [ ]:
colours = plt.cm.viridis(np.linspace(0,1,10))
golden_ratio = (1 + np.sqrt(5)) / 2

In [ ]:
bound = 4
body_buffer = 0.14
head_buffer = body_buffer * golden_ratio
head_stretch = 1.3

expected_x_bounds = np.array([-bound - body_buffer, bound + head_buffer])
length = np.diff(expected_x_bounds)
height = length / golden_ratio
desired_amplitude = height / 2 - body_buffer

def snake_wave_packet(x):
    x = np.array(x)
    return desired_amplitude * np.exp(-x**2/2) * np.sin(2 * np.pi * x) 

expected_y_bounds = np.ravel([-height/2, height/2])

assert np.diff(expected_x_bounds) / np.diff(expected_y_bounds) == golden_ratio

In [ ]:
epsilon = 0.1
x_init = bound - head_buffer * head_stretch

y_vals = snake_wave_packet([x_init, x_init + epsilon])
ydiff = np.diff(y_vals)
y_init = y_vals[0]

head_shift = head_buffer * head_stretch
hypot = np.sqrt(epsilon**2 + ydiff**2)
num_shifts = head_shift / hypot
neck_shring = 1

head_angle = np.arctan(ydiff / epsilon) / np.pi * 180
head_middle_x = x_init + epsilon * num_shifts * neck_shring
head_middle_y = y_init + ydiff * num_shifts * neck_shring

In [ ]:
tongue_length = 0.1
tongue_buffer = 0.02
fork_angles = 35

tongue_init_x = head_middle_x + epsilon * num_shifts
tongue_init_y = head_middle_y + ydiff * num_shifts

tongue_shift_num = tongue_length / hypot

tongue_shift_x = epsilon * tongue_shift_num
tongue_shift_y = ydiff * tongue_shift_num

tongue_fork_x = tongue_init_x + tongue_shift_x
tongue_fork_y = tongue_init_y + tongue_shift_y

tongue_base = geo.LineString(
    [(tongue_init_x, tongue_init_y), (tongue_fork_x, tongue_fork_y)]
).buffer(tongue_buffer)

tongue_base

tongue_top_fork = aff.rotate(tongue_base, fork_angles, origin=(tongue_init_x, tongue_init_y))
tongue_top_fork = aff.translate(tongue_top_fork, tongue_shift_x, tongue_shift_y)

tongue_bot_fork = aff.rotate(tongue_base, -fork_angles, origin=(tongue_init_x, tongue_init_y))
tongue_bot_fork = aff.translate(tongue_bot_fork, tongue_shift_x, tongue_shift_y)


tongue = tongue_base.union(tongue_top_fork).union(tongue_bot_fork)
tongue

In [ ]:
# head_angle

In [ ]:
# ydiff

In [ ]:
# np.arctan(ydiff / epsilon) / np.pi * 180

In [ ]:
point = geo.Point(head_middle_x, head_middle_y)
head = point.buffer(head_buffer)
head = aff.scale(head, head_stretch, 1, origin=point)
head = aff.rotate(head, head_angle, origin=point)
head = head.union(tongue)

head

In [ ]:
x = np.linspace(-bound, x_init, 1000)
y = snake_wave_packet(x)
linestring = geo.LineString(zip(x, y))
body = linestring.buffer(body_buffer)
body

In [ ]:
snake = body.union(head)
snake

In [ ]:
xmin, ymin, xmax, ymax = snake.bounds

In [ ]:
def create_splotch(snake, body_buffer):
    xmin, ymin, xmax, ymax = snake.bounds
    x = np.random.uniform(xmin, xmax)
    y = np.random.uniform(ymin, ymax)
    size = np.random.uniform(body_buffer/2, body_buffer*5)
    
    circle = geo.Point(x, y).buffer(size)
    
    if circle.intersects(snake):
        return circle.intersection(snake)
    else:
        return create_splotch(snake, body_buffer)

In [ ]:
num_splotches = 100

splotches = [
    create_splotch(snake, body_buffer)
    for i in range(num_splotches)
]

In [ ]:
splotches_x = np.random.uniform(xmin, xmax, size=num_splotches)
splotches_y = np.random.uniform(ymin, ymax, size=num_splotches)

splotch_size = np.random.uniform(body_buffer/10, body_buffer*2, size=num_splotches)

# splotches = [
#     geo.Point(xi, yi).buffer(size).intersection(snake) for xi, yi, size in zip(splotches_x, splotches_y, splotch_size)
# ]

splotch_colours = colours[np.random.choice(np.arange(len(colours)), size=num_splotches)]

# splotch_colour

In [ ]:
# splotches[2]

In [ ]:
def plot_bounds():
    plt.plot([expected_x_bounds[0], expected_x_bounds[1]], [expected_y_bounds[0], expected_y_bounds[0]], 'k', linewidth=0.5)
    plt.plot([expected_x_bounds[0], expected_x_bounds[1]], [expected_y_bounds[1], expected_y_bounds[1]], 'k', linewidth=0.5)
    
    plt.plot([expected_x_bounds[0], expected_x_bounds[0]], [expected_y_bounds[0], expected_y_bounds[1]], 'k', linewidth=0.5)
    plt.plot([expected_x_bounds[1], expected_x_bounds[1]], [expected_y_bounds[0], expected_y_bounds[1]], 'k', linewidth=0.5)

In [ ]:
fig = plt.figure(1, figsize=(10,10), dpi=90)
fig.set_frameon(True)
ax = fig.add_subplot(111)

patch = PolygonPatch(snake, facecolor=colours[6])
ax.add_patch(patch)

plt.xlim(-bound-1, bound+1)
plt.ylim(-bound-1, bound+1)

for splotch, splotch_colour in zip(splotches, splotch_colours):
    patch = PolygonPatch(splotch, facecolor=splotch_colour)
    ax.add_patch(patch)

#     plot_bounds()

#     plt.plot(x,y, color='white')

plt.show()

In [ ]:
# for colour in colours:
#     fig = plt.figure(1, figsize=(10,10), dpi=90)
#     fig.set_frameon(True)
#     ax = fig.add_subplot(111)

#     patch = PolygonPatch(snake, facecolor=colour)
#     ax.add_patch(patch)

#     plt.xlim(-bound-1, bound+1)
#     plt.ylim(-bound-1, bound+1)
    
#     for splotch, splotch_colour in zip(splotches, splotch_colours):
#         patch = PolygonPatch(splotch, facecolor=splotch_colour)
#         ax.add_patch(patch)
    
# #     plot_bounds()
    
# #     plt.plot(x,y, color='white')
    
#     plt.show()